In [ ]:
# https://medium.com/@kevinossner/how-much-can-you-deadlift-96f84ca5f3ee

In [ ]:
# https://www.openpowerlifting.org./rankings/raw/fully-tested/men/2021

In [1]:
import numpy as np
import pandas as pd
import urllib.request as urllib
from bs4 import BeautifulSoup
import requests


import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette(sns.diverging_palette(220, 20, n=7))
%matplotlib inline
import time
import random

import re
# df = pd.read_csv(fname, parse_dates=True)

In [2]:
def lbs_to_kg(lbs):
    return lbs * 0.453592

def kg_to_lbs(kg):
    return kg * 2.20462

In [4]:
def get_soup(url):
    """ Returns the BeautifulSoup object for website of the given category name in the
        given CL webpage's homepage

    Args:
        url(String): the given URL

    Returns:
        soup(bs4.BeautifulSoup): the BeautifulSoup object representation of the desired category page
    """

    # Open the target category page
    # html = urllib.urlopen(url)
    html = requests.get(url)
    # Create a BeautifulSoup object after the HTML page is read
    # soup = BeautifulSoup(html.read())
    soup = BeautifulSoup(html.content, "html.parser")

    # Close the urllib connection to avoid issues with the website
    html.close()

    return soup

In [5]:
def get_dict(sex: str='M'):
    url = ''
    min_class = 0
    max_class = 0
    
    if sex == 'M':
        url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid=122663'
        min_class = 52.0
        max_class = 141.0
    else:
        url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120362&categoryid=59&weightclassid=122653'
        min_class = 44.0
        max_class = 101.0
        
    soup = get_soup(url)
    temp = soup.find('select', attrs={'name': 'weightclassid'})
    choices = temp.find('optgroup')
    options = choices.find_all('option')
    url_class_map = {}
    for option in options:
        w_class = ''
        # Handle max class
        if option.get_text()[-1] == '+':
            w_class = max_class
        else:
            w_class = float(option.get_text()) * -1
        if w_class >= min_class:
            url = option['value']
            url_class_map[url] = w_class
    print(url_class_map)
    return url_class_map

get_dict('M')

{'122643': 52.0, '122663': 56.0, '122644': 60.0, '122645': 67.5, '122646': 75.0, '122647': 82.5, '122648': 90.0, '122649': 100.0, '122650': 110.0, '122651': 125.0, '122673': 140.0, '122671': 141.0}


{'122643': 52.0,
 '122663': 56.0,
 '122644': 60.0,
 '122645': 67.5,
 '122646': 75.0,
 '122647': 82.5,
 '122648': 90.0,
 '122649': 100.0,
 '122650': 110.0,
 '122651': 125.0,
 '122673': 140.0,
 '122671': 141.0}

In [6]:
def get_records_df():
    m_dict = get_dict('M')
    f_dict = get_dict('F')
    dicts = [(m_dict, 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120365&categoryid=59&weightclassid=', 'M'),
             (f_dict, 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120362&categoryid=59&weightclassid=', 'F')]
    # Create DataFrame
    df = pd.DataFrame(columns = ['Name', 'Weight Class', 'Lift', 'Weight (kg)', 'Date', 'Sex'])
    
    for info in dicts:
        info_dict = info[0]
        target_url = info[1]
        sex = info[2]
        for url, w_class in info_dict.items():
            page = f'{target_url}{url}'
            soup = get_soup(page)
            body = soup.find_all('tbody')[1]
            # print(body)
            # titles = body.find_all('th', {'colspan' : '9'})
            records = body.find_all('tr', class_=None)
            lift = ''
            for count, record in enumerate(records):
                # even tags are lift title
                if count > 0 and (count-1)%2 == 0:
                    lift = record.get_text().strip()
            #         print(lift)
            #         print(record.get_text().strip())
                # odd tags are info
                if count > 0 and (count-1)%2 == 1:
                    td_tags = record.find_all('td')
                    name = td_tags[1]
#                     weight_class = td_tags[0]
                    weight = td_tags[2]
                    date = td_tags[3]
                    # Create a temporary dictionary to store the information of the current post
                    temp_dict = {'Name': name, 'Weight Class': w_class, 'Lift': lift, 'Weight (kg)': weight, 'Date': date, 'Sex': sex}
                    # Append the current post's information to the df DataFrame to create its respective row
                    df = df.append(temp_dict, ignore_index=True)
            #         for info in infos:
            #       except  print(info.get_text().strip())
            print('Running...')
            time.sleep(random.randint(2,3))
    
    # Wrangling
    df['Name'] = df['Name'].apply(lambda x: x.get_text())
    df['Weight (kg)'] = df['Weight (kg)'].apply(lambda x: float(x.get_text().split()[0]) if len(x.get_text())>0 else np.nan)
    df['Date'] = df['Date'].apply(lambda x: re.sub(r"[\n\t\s]*", "", x.get_text()))
    return df

In [7]:
df = get_records_df()
df

{'122643': 52.0, '122663': 56.0, '122644': 60.0, '122645': 67.5, '122646': 75.0, '122647': 82.5, '122648': 90.0, '122649': 100.0, '122650': 110.0, '122651': 125.0, '122673': 140.0, '122671': 141.0}
{'122653': 44.0, '122654': 48.0, '122655': 52.0, '122656': 56.0, '122657': 60.0, '122658': 67.5, '122659': 75.0, '122660': 82.5, '122661': 90.0, '122672': 100.0, '122670': 101.0}
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...
Running...


,Name,Weight Class,Lift,Weight (kg),Date,Sex
0,Jasen Hinkel,52.0,Raw Open - Squat,165.0,07/01/2008,M
1,Caleb Tourres,52.0,Raw Open - Bench press,100.0,07/01/2008,M
2,Caleb Tourres,52.0,Raw Open - Bench press single lift,100.0,07/01/2008,M
3,Sean O'Leary,52.0,Raw Open - Deadlift,115.0,07/01/2008,M
4,To be set,52.0,Raw Open - Deadlift single lift,NaN,,M
...,...,...,...,...,...,...
133,Mahailya Reeves,101.0,Raw Open - Bench press,167.5,10/16/2019,F
134,Mahailya Reeves,101.0,Raw Open - Bench press single lift,167.5,10/16/2019,F
135,Sarah Brenner,101.0,Raw Open - Deadlift,260.0,10/16/2019,F
136,Sarah Brenner,101.0,Raw Open - Deadlift single lift,260.0,10/16/2019,F


In [16]:
target_url = 'https://usapl.liftingdatabase.com/records-default?recordtypeid=120362&categoryid=59&weightclassid='

df2 = pd.DataFrame(columns = ['Name', 'Weight Class', 'Lift', 'Weight (kg)', 'Date'])

for url, w_class in url_class_map.items():
    page = f'{target_url}{url}'
    soup = get_soup(page)
    body = soup.find_all('tbody')[1]
    # print(body)
    # titles = body.find_all('th', {'colspan' : '9'})
    records = body.find_all('tr', class_=None)
    lift = ''
    for count, record in enumerate(records):
        # even tags are lift title
        if count > 0 and (count-1)%2 == 0:
            lift = record.get_text().strip()
    #         print(lift)
    #         print(record.get_text().strip())
        # odd tags are info
        if count > 0 and (count-1)%2 == 1:
            infos = record.find_all('td')
            name = infos[1]
            weight_class = infos[0]
            weight = infos[2]
            date = infos[3]
            # Create a temporary dictionary to store the information of the current post
            temp_dict = {'Name': name, 'Weight Class': w_class, 'Lift': lift, 'Weight (kg)': weight, 'Date': date}
            # Append the current post's information to the df DataFrame to create its respective row
            df2 = df2.append(temp_dict, ignore_index=True)
    #         for info in infos:
    #       except  print(info.get_text().strip())
    time.sleep(random.randint(2,3))

df2

,Name,Weight Class,Lift,Weight (kg),Date
0,[[Stephanie Rattunde]],44.0,Raw Open - Squat,[85 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
1,[[Stephanie Rattunde]],44.0,Raw Open - Bench press,[62.5 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
2,[[Stephanie Rattunde]],44.0,Raw Open - Bench press single lift,[62.5 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
3,[[Stephanie Rattunde]],44.0,Raw Open - Deadlift,[125 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
4,[[Stephanie Rattunde]],44.0,Raw Open - Deadlift single lift,[125 kg],[\n\t\t\t \t\t\t\t10/15/2015\n\t\t\t \t\...
...,...,...,...,...,...
61,[[Mahailya Reeves]],101.0,Raw Open - Bench press,[167.5 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
62,[[Mahailya Reeves]],101.0,Raw Open - Bench press single lift,[167.5 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
63,[[Sarah Brenner]],101.0,Raw Open - Deadlift,[260 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...
64,[[Sarah Brenner]],101.0,Raw Open - Deadlift single lift,[260 kg],[\n\t\t\t \t\t\t\t10/16/2019\n\t\t\t \t\...


In [8]:
from datetime import datetime
# datetime.strptime('2014-12-04', '%Y-%m-%d').date()
# datetime.strptime('10/10/2020', '%m/%d/%Y').date()

df['Date'] = df['Date'].apply(lambda x: datetime(1, 1, 1).date() if len(x) == 0 else datetime.strptime(x, '%m/%d/%Y').date())
df
# datetime.date(2014, 12, 4)

,Name,Weight Class,Lift,Weight (kg),Date,Sex
0,Jasen Hinkel,52.0,Raw Open - Squat,165.0,2008-07-01,M
1,Caleb Tourres,52.0,Raw Open - Bench press,100.0,2008-07-01,M
2,Caleb Tourres,52.0,Raw Open - Bench press single lift,100.0,2008-07-01,M
3,Sean O'Leary,52.0,Raw Open - Deadlift,115.0,2008-07-01,M
4,To be set,52.0,Raw Open - Deadlift single lift,NaN,0001-01-01,M
...,...,...,...,...,...,...
133,Mahailya Reeves,101.0,Raw Open - Bench press,167.5,2019-10-16,F
134,Mahailya Reeves,101.0,Raw Open - Bench press single lift,167.5,2019-10-16,F
135,Sarah Brenner,101.0,Raw Open - Deadlift,260.0,2019-10-16,F
136,Sarah Brenner,101.0,Raw Open - Deadlift single lift,260.0,2019-10-16,F


In [13]:
df.groupby(['Lift', 'Weight Class']).first()
# df.groupby(['Lift']).get_group('Raw Open - Deadlift')

Name  Weight (kg)  \
Lift                   Weight Class                                    
Raw Open - Bench press 44.0          Stephanie Rattunde         62.5   
                       48.0             Demetria Thaten         90.0   
                       52.0               Caleb Tourres        100.0   
                       56.0                Dalton Lacoe        115.0   
                       60.0              Shaheed Bryant        152.0   
...                                                 ...          ...   
Raw Open - Total       101.0               Bonica Brown        671.5   
                       110.0              Ashton Rouska        955.0   
                       125.0           Dennis Cornelius        978.5   
                       140.0                  To be set          NaN   
                       141.0               Ray Williams       1112.5   

                                           Date Sex  
Lift                   Weight Class                  
Raw Open - Bench press 44.0          2015-10-15   F  
                       48.0          2021-06-14   F  
                       52.0          2008-07-01   M  
                       56.0          2015-06-05   M  
                       60.0          2021-06-14   M  
...                                         ...  ..  
Raw Open - Total       101.0         2018-06-06   F  
                       110.0         2021-12-04   M  
                       125.0         2016-06-18   M  
                       140.0         0001-01-01   M  
                       141.0         2019-03-02   M  

[90 rows x 4 columns]

In [12]:
type(df['Date'][0])

str

In [23]:
total_data = df.append(df2, ignore_index=True)

In [9]:
df.to_csv('current_usapl_american_raw_records.csv', index=False)

In [47]:
df[(df['Weight Class'] == 52.0) & (df['Lift'] == 'Raw Open - Squat')]

,Name,Weight Class,Lift,Weight (kg),Date,Sex
0,Jasen Hinkel,52.0,Raw Open - Squat,165.0,2008-07-01,M
84,Suzanne Hartwig-Gary,52.0,Raw Open - Squat,156.5,2017-06-15,F
